In [ ]:
!git clone https://github.com/yandproject/CycleGAN-cartoon-reenactment

In [ ]:
import os

os.chdir('CycleGAN-cartoon-reenactment/')

In [ ]:
from face_parsing.logger import setup_logger
from face_parsing.model import BiSeNet

import torch

import os
import os.path as osp
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import cv2

In [ ]:
 from google.colab import drive 
 drive.mount("/content/drive")

Mounted at /content/drive


test

In [ ]:
def vis_parsing_maps(im, parsing_anno, stride, save_im=False, save_path='vis_results/parsing_map_on_im.jpg'):
    # Colors for all 20 parts
    part_colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0],
                   [255, 0, 85], [255, 0, 170],
                   [0, 255, 0], [85, 255, 0], [170, 255, 0],
                   [0, 255, 85], [0, 255, 170],
                   [0, 0, 255], [85, 0, 255], [170, 0, 255],
                   [0, 85, 255], [0, 170, 255],
                   [255, 255, 0], [255, 255, 85], [255, 255, 170],
                   [255, 0, 255], [255, 85, 255], [255, 170, 255],
                   [0, 255, 255], [85, 255, 255], [170, 255, 255]]

    im = np.array(im)
    vis_im = im.copy().astype(np.uint8)
    vis_parsing_anno = parsing_anno.copy().astype(np.uint8)
    vis_parsing_anno = cv2.resize(vis_parsing_anno, None, fx=stride, fy=stride, interpolation=cv2.INTER_NEAREST)
    vis_parsing_anno_color = np.zeros((vis_parsing_anno.shape[0], vis_parsing_anno.shape[1], 3)) + 255

    num_of_class = np.max(vis_parsing_anno)

    for pi in range(1, num_of_class + 1):
        index = np.where(vis_parsing_anno == pi)
        vis_parsing_anno_color[index[0], index[1], :] = part_colors[pi]

    vis_parsing_anno_color = vis_parsing_anno_color.astype(np.uint8)
    # print(vis_parsing_anno_color.shape, vis_im.shape)
    vis_im = cv2.addWeighted(cv2.cvtColor(vis_im, cv2.COLOR_RGB2BGR), 0.4, vis_parsing_anno_color, 0.6, 0)

    # Save result or not
    if save_im:
        cv2.imwrite(save_path[:-4] +'.png', vis_parsing_anno)
        cv2.imwrite(save_path, vis_im, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

    # return vis_im

def evaluate(respth='/content/drive/MyDrive/Results/', dspth='./data', cp='model_final_diss.pth'):

    if not os.path.exists(respth):
        os.makedirs(respth)

    n_classes = 19
    net = BiSeNet(n_classes=n_classes)
    net.cuda()
    save_pth = osp.join('/content/drive/MyDrive/Results/', cp)
    net.load_state_dict(torch.load(save_pth))
    net.eval()

    to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    with torch.no_grad():
        for image_path in os.listdir(dspth):
            img = Image.open(osp.join(dspth, image_path))
            image = img.resize((256, 256), Image.BILINEAR)
            img = to_tensor(image)
            img = torch.unsqueeze(img, 0)
            img = img.cuda()
            out = net(img)[0]
            parsing = out.squeeze(0).cpu().numpy().argmax(0)
            # print(parsing)
            print(np.unique(parsing))

            vis_parsing_maps(image, parsing, stride=1, save_im=True, save_path=osp.join(respth, image_path))


You can download pre-trained model for face parsing [here](https://drive.google.com/file/d/154JgKpzCPW82qINcVieuPH3fZ2e0P812/view)

In [ ]:
#test
if __name__ == "__main__":
    evaluate(dspth='/content/drive/MyDrive/dataset/', cp=' 'your path to the file'/79999_iter.pth')

In [ ]:
img = Image.open('/content/drive/MyDrive/Results/'filename'.png')

In [ ]:
l = []

In [ ]:
for i in range(0, 256):
  for j in range(0, 256):
    l.append((img.convert('RGB').getpixel((j, i))[0]))

In [ ]:
mask = []

In [ ]:
for i in range(0, 256*256):
  if l[i] in [4, 5, 6, 11, 12, 13]:
    mask.append(8)
  else:
    mask.append(1)

In [ ]:
mask = np.resize(mask, (256, 256))

In [ ]:
ones = []

In [ ]:
for i in range(0, 256):
  ones.append(1)

In [ ]:
w = []

In [ ]:
for i in range(0, 256):
  for j in range(0, 256):
    ww = []
    if mask[i][j] != 1:
      ww.append(i)
      ww.append(j)
      ww.append(mask[i][j])
      w.append(ww)

In [ ]:
cuda0 = torch.device('cuda:0')
a = torch.ones([1, 3, 256, 256], dtype=torch.float, device=cuda0)

for element in w:
    i = element[0]
    j = element[1]
    weight = element[2]
    
    a[0][0][i][j] = weight
    a[0][1][i][j] = weight
    a[0][2][i][j] = weight

weights_1 = a

In [ ]:
torch.save(weights_1, 'weights_1.pt')

In [ ]:
cuda0 = torch.device('cuda:0')

b = torch.ones([1, 1, 256, 256], dtype=torch.float, device=cuda0)

for element in w:
    i = element[0]
    j = element[1]
    weight = element[2]
    
    b[0][0][i][j] = weight
weights_0 = b

In [ ]:
torch.save(weights_0, 'weights_0.pt')